# Tradesman

Tradesman is a friendly model builder for transportation models. 

On this example, we show how to create a transportation model for Gibraltar, a small british territory in mainland Europe. In the end of the model-building process, we also have some simple data visualization.

## Running on Google Colab

A few steps need to be taken after pressing the button below

<a href="https://colab.research.google.com/github/outerl/tradesman-demo/blob/main/model_building_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

1. Uncomment and run cell below
2. re-start your runtime (Runtime-Restart Runtime or ctrl+m)
3. Start again, but from the **Import package** section

In [ ]:
# !apt-get update && apt-get install libsqlite3-mod-spatialite
# !apt-get install -y libspatialite-dev
# !pip install "numpy>1.22,<1.24" --upgrade
# !pip install tradesman

## Import package

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from uuid import uuid4
from tradesman.model import Tradesman

c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.


## Create model

We're going to save our model in the *temp* folder

If you have are running this notebook on Google Colab and want to save your model into your Google Drive, you should:

1. Un-comment and run the cell below
2. Accept the terms and conditions, and replace `"gettempdir()"` with `"/content/gdrive"`.


In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
from tempfile import gettempdir

model_place = "Gibraltar"
folder = os.path.join(gettempdir(), uuid4().hex)

In [ ]:
model = Tradesman(network_path=folder, model_place=model_place)

We could just create the model with default parameters by calling  `model.create()`, but we want to make all components explicit

In [ ]:
%%time
# Import the model area
model.import_model_area()

# The borders for the country that includes this modeling area
model.add_country_borders()

# Imports the political subdivisions for this region
model.import_subdivisions(subdivision_levels=2, overwrite=True)

In [ ]:
%%time
# Import the model network
model.import_network()

In [ ]:
%%time
# Import population
model.import_population()

As Monaco has a really small territorial area, we'll create Traffic Analysis Zones (TAZs) more suitable sizes to the country rather than the default ones.

In [ ]:
# Build TAZs with zones in the range of 100 to 500 people
# The default range is 500 to 10,000
# The `hexbin_size` parameter is the size of the hexbin side in meters.  Very small hexbins make the computation much slower, but may be necessary if you population targets are small.  The default parameter is 200
model.build_zoning(hexbin_size=100, min_zone_pop=100, max_zone_pop=500)

In [ ]:
%%time
# Imports the population pyramid for our new zone system
model.import_pop_by_sex_and_age()

In [ ]:
%%time
# Import amenities from Open-Street Maps
model.import_amenities()

In [ ]:
# Import buildings
model.import_buildings(download_from_bing=False)

## Create a synthetic population

Let's create our synthetic population for Gibraltar! It's a two-step process: first, we sample a population dataset to create our seed sample that resembles Gibraltar (seed creation) and then, we run the synthesizer.
The sample that ships with the software is a sample for Texas, and the software is separated in the creation of the PopulationSim configuration and the actual population generation to allow users to use a more appropriate sample if available

In [ ]:
# Create sample to build synthetic population
model.build_population_synthesizer_data(sample_size=0.02)

From the output, we can visualize if there are more or fewer households of certain categories. It's important to highlight that differences between expected and found values are expected because the data used to create the sample doesn't belong to Monaco.

In [ ]:
# Create synthetic population
model.synthesize_population()

After the execution of the above cell, there are two main outputs. The ones related to non-controlled or controlled variables. The first one represents the differences between expected and found values for non-controlled variables based on United Nation Household survey data. The graphs represent the percentage difference between the synthetic over the expected population for controlled variables.

## And as promised, a bit of data visualization

### Import libraries

In [ ]:
import geopandas as gpd
import folium

### Layer set up

In [ ]:
colors = ["#219EBC", "#ffb703", "#8ECAE6", "#023047", "#fb8500"]
location = [36.142260, -5.346601] # This is the location of the Monte Carlo Casino, in case you're curious

In [ ]:
# Let's create a variable with our project database connection
cnx = model._project.conn

### Plot the network

In [ ]:
query = "SELECT link_type, distance, modes, ST_AsBinary(geometry) geom FROM links;"
links = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)

# We'll plot only some link types
links = links[links.link_type.isin(["residential", "primary", "secondary", "tertiary"])]

In [ ]:
m = None

for idx, tp in enumerate(links.link_type.unique()):
  gdf = links[links.link_type == tp]
  if m:
    gdf.explore(m=m, name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                  zoom_start=14, location=location, legend=False, color=colors[idx])
  else:
    m = gdf.explore(name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                      zoom_start=14, location=location, legend=False, color=colors[idx])

folium.LayerControl().add_to(m)

m

### Plot the populational density


In [ ]:
query = "SELECT *, ST_AsBinary(geometry) geom FROM zones;"
zones = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)
zones.drop(columns=["geometry"], inplace=True)

# Let's create our populational densisity variable
zones["POP_DENSITY"] = zones["population"] / (zones["geom"].to_crs(3857).area * 10e-6)


In [ ]:
zones.explore("POP_DENSITY", tiles="CartoDB positron", cmap="Blues", tooltip=False,
              style_kwds={"fillOpacity": 1.0}, zoom_start=14, location=location, popup=True)


In [ ]:
model.close()